# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
!pip install textdistance

In [2]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

import textdistance

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [3]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    # вся эффективноть берется из того, что мы сразу считаем близость 
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    max_closest_prior = max(closest, key=lambda x: x[1])[1]
    closest_candidates = [(cand[0], P(cand[0])) for cand in closest if cand[1] == max_closest_prior]
    
    return max(closest_candidates, key=lambda x: x[1])[0]

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [4]:
corpus = open('data\wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000)
X = vec.fit_transform(vocab)

In [5]:
N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

In [6]:
get_closest_hybrid_match('жзть', X, vec)

'жить'

Суть доработки состоит в том, что из списка с кандидатами с расстоянием редактирования получается максимальное значение. По нему происходит фильтрация кандидатов. Затем у каждого кандидата вычисляется вероятность встречи в корпусе и среди них после выбирается тот кандидат, который имеет наибольшую вероятность.

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [7]:
WORDS = list(vocab.keys())

In [8]:
def get_deletes(word):
    deletes = [word[:i] + word[i + 1:]  for i in range(len(word))]
    deletes.append(word)
    return deletes

In [9]:
def known(words):
    return set(w for w in words if w in WORDS)

In [10]:
def candidates(word):
    return (known([word]) or known(correction_variants(word)) or [word])

In [11]:
def correction(word):
    return max(candidates(word), key=P)

In [12]:
def correction_variants(word):
    errors = get_deletes(word)
    
    set_corr = set()

    for error in errors:
        set_corr = set_corr or set(dict_corr.get(error, ''))
    
    return set_corr

In [13]:
def eval_mistakes():
    correct = 0
    total = 0

    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    cashed = {}
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            # чтобы два раза не исправлять одно и тоже слово - закешируем его
            # перед тем как считать исправление проверим нет ли его в кеше

            predicted = cashed.get(pair[1], correction(pair[1]))
            cashed[pair[1]] = predicted


            if predicted == pair[0]:
                correct += 1
            total += 1

            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] !=  predicted:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == predicted:
                    mistaken_fixed += 1

        if not i % 100:
            print(i)
            
    print(correct/total)
    print(mistaken_fixed/total_mistaken)
    print(correct_broken/total_correct)

In [14]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [15]:
dict_corr = {}

for word in WORDS:
    if len(word) > 1:
        deletes = get_deletes(word)
        for delete in deletes:
            if delete in dict_corr:
                dict_corr[delete].append(word)
            else:
                dict_corr[delete] = [word]

In [16]:
bad = open('data\sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data\correct_sents.txt', encoding='utf8').read().splitlines()

In [17]:
eval_mistakes()

0
100
200
300
400
500
600
700
800
900
0.8680340170085042
0.327639751552795
0.05202710462845986


In [18]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [19]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'ясеня'),
 ('вообще', 'вобще', 'общей'),
 ('вообще', 'ваще', 'чаще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'статей'),
 ('очень', 'ооочень', 'ооочень'),
 ('как-то', 'както', 'актов'),
 ('очень', 'оооочень', 'оооочень'),
 ('это', 'ето', 'что')]

In [20]:
%%time
correction('солнвце')

Wall time: 26 ms


'солнце'

In [21]:
%%time
correction('насмехатьсяаававттававаываываы')

Wall time: 14 ms


'насмехатьсяаававттававаываываы'

По качеству работы алгоритм Symspell уступает алгоритму Норвига, хотя работает он  не сильно хуже. Главным же преимуществом является его скорость работы из-за использования только одной операции удаления.

## *3. Настройка гиперпараметров. (2 балла)

У метода из первого заданий много гиперпараметров (те которые нужно подбирать самостоятельно). Это параметры векторайзера, topn, метрика редактирования. Поэкспериментируйте с ними. 

Проведите как минимум 10 экспериментов с разными параметрами. Для каждого эксперимента укажите мотивацию (например, "слишком маленький topn в get_closest_match_vec приводит к тому, что некоторые хорошие варианты не доходят до get_closest_match_with_metric, попробуем его увеличить")

Старайтесь получить улучшение, но если улучшить не получится, то это не страшно. Главное, чтобы эксперименты были осмысленными, а не рандомными. 

In [22]:
def eval_mistakes_closest(X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    mistakes = []
    total_mistaken = 0
    mistaken_fixed = 0

    total_correct = 0
    correct_broken = 0

    total = 0
    correct = 0

    cashed = {}
    for i in range(len(true)):
        word_pairs = align_words(true[i], bad[i])
        for pair in word_pairs:
            if predict_mistaken(pair[1], vocab):
                pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec, topn=topn, metric=metric))
                cashed[pair[1]] = pred
            else:
                pred = pair[1]


            if pred == pair[0]:
                correct += 1
            else:
                mistakes.append((pair[0], pair[1], pred))
            total += 1

            if pair[0] == pair[1]:
                total_correct += 1
                if pair[0] != pred:
                    correct_broken += 1
            else:
                total_mistaken += 1
                if pair[0] == pred:
                    mistaken_fixed += 1

        if not i % 100:
            print(i)
    
    print(correct/total)
    print(mistaken_fixed/total_mistaken)
    print(correct_broken/total_correct)

In [23]:
%%time
#base

vec = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=1000, )
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec)

0
100
200
300
400
500
600
700
800
900
0.8556278139069535
0.48835403726708076
0.09004249454461927
Wall time: 8min 16s


In [24]:
%%time
# Имеет смысл увеличить параметр н-грамм, 
# тем самым увеличится количество возможных ошибок на слов и возможно увеличится качество.

vec = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=1000, )
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec)

0
100
200
300
400
500
600
700
800
900
0.8556278139069535
0.48835403726708076
0.09004249454461927
Wall time: 7min 58s


In [25]:
%%time
# Попробуем уменьшить количество фич, чтобы сократить время работы, вероятно это никак не повлияет на качество,
# потому что 300 фич может быть достаточно для обучения.

vec = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=300, )
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec)

0
100
200
300
400
500
600
700
800
900
0.8544272136068034
0.47903726708074534
0.09004249454461927
Wall time: 6min 31s


In [26]:
%%time
# Попробуем увеличить значение параметра topn, возможно в сформированном списке косинусного расстояния найдётся лучший кандидат.

vec = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=300, )
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec, topn=15)

0
100
200
300
400
500
600
700
800
900
0.8577288644322161
0.5046583850931677
0.09004249454461927
Wall time: 6min 30s


In [27]:
%%time
# Попробуем поменять метрику на метрику Левенштейна, поскольку в ней происходит меньше операций, чем у Левенштейна-Дамерау,
# потому, вероятно, алгоритм будет работать быстрее.

vec = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=300, )
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec, topn=15, metric=textdistance.levenshtein)

0
100
200
300
400
500
600
700
800
900
0.856328164082041
0.4937888198757764
0.09004249454461927
Wall time: 6min 51s


In [28]:
%%time
# Попробуем снизить метрику max_df, потому что она может пропускать важные фичи, 
# а поскольку мы работаем с символами, то это может оказаться критично.

vec = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=300, max_df=0.3, )
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec, topn=15, metric=textdistance.levenshtein)

0
100
200
300
400
500
600
700
800
900
0.8554277138569285
0.48680124223602483
0.09004249454461927
Wall time: 4min 55s


In [29]:
%%time
# Попробуем установить метрику min_df в качестве 0.05, потому что такое значение будет игнорировать те фичи, которые будут
# специфичны (например, какие-то специальные символы).

vec = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=300, max_df=0.3, min_df=0.05)
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec, topn=15, metric=textdistance.levenshtein)

0
100
200
300
400
500
600
700
800
900
0.8498249124562282
0.44332298136645965
0.09004249454461927
Wall time: 3min 10s


In [30]:
%%time
# Попробуем поменять параметр 'analyzer' на 'char_wb', потому что, исходя из документации, 
# будут создаваться n-граммы внутри слова, что, возможно, может снизить частоту каких-нибудь специальных символов. 

vec = CountVectorizer(analyzer='char_wb', ngram_range=(1,4), max_features=300, max_df=0.3, min_df=0.05)
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec, topn=15, metric=textdistance.levenshtein)

0
100
200
300
400
500
600
700
800
900
0.8545272636318159
0.4798136645962733
0.09004249454461927
Wall time: 3min 25s


In [31]:
%%time
# Имеет смысл попробовать другой векторайзер - TfidfVectorizer, поскольку в предыдущей лабораторной он показал себя лучше.

vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(1,4), max_features=300, max_df=0.3, min_df=0.05)
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec, topn=15, metric=textdistance.levenshtein)

0
100
200
300
400
500
600
700
800
900
0.8534267133566783
0.47127329192546585
0.09004249454461927
Wall time: 3min 5s


In [32]:
%%time
# Похоже, что TfidfVectorizer работает хуже, некоторые параметры не дают прироста качества и скорости. 
# Попробуем взять те параметры, которые улучшали модель, и те, с которых начинался бейзлайн. 
# В частности, можно дальше попробовать увеличить topn,
# потому что гипотеза оказалась верна и некоторые исправления также могут не доходить.
# Также изменение метрики помогло увеличить скорость работы алгоритма.

vec = CountVectorizer(analyzer='char_wb', ngram_range=(1,3), max_features=1000, )
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec, topn=25, metric=textdistance.levenshtein)

0
100
200
300
400
500
600
700
800
900
0.8562281140570285
0.49301242236024845
0.09004249454461927
Wall time: 9min 47s


In [33]:
%%time
# Предположим, что нам не важна частотность у векторайзера, поскольку будет достаточно знать, 
# есть ли символ или n-грамма в слове, и результат косинусного расстояния может оказаться лучше. 
# В таком случае надо изменить параметр binary у векторайзера на True. 

vec = CountVectorizer(analyzer='char_wb', ngram_range=(1,3), max_features=1000, binary=True)
X = vec.fit_transform(vocab)

eval_mistakes_closest(X, vec, topn=25, metric=textdistance.levenshtein)

0
100
200
300
400
500
600
700
800
900
0.8566283141570785
0.49611801242236025
0.09004249454461927
Wall time: 9min 51s


В результате получилось улучшить качество работы алгоритма, но это улучшение по факту не совсем значительное, примерно на сотые или тысячные доли. Ещё при некоторых запусках получилось увеличить скорость работы, но иногда алгоритм работает и дольше стартовых параметров, скорее всего это зависит от нагруженности процессора. 